In [93]:
!pip install pytorch_pretrained_bert

In [94]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [95]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [96]:
%cd "/content/drive/My Drive/Gatech/spring_20/CS4650_NLP/project"
!pwd
!ls 

/content/drive/My Drive/Gatech/spring_20/CS4650_NLP/project
/content/drive/My Drive/Gatech/spring_20/CS4650_NLP/project
 bert_model_test.pth   'IMDB Dataset.csv'	   train.csv
 bert_train_data.csv    IMDB_Dataset_SCRATCH.csv
 clean_train_data.csv   test.csv


In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from random import randrange
import torch.nn.functional as F

In [0]:
text ='what is a pug'

In [0]:
zz = tokenizer.tokenize(text)


In [0]:
class BertLayerNorm(nn.Module):
        def __init__(self, hidden_size, eps=1e-12):
            """Construct a layernorm module in the TF style (epsilon inside the square root).
            """
            super(BertLayerNorm, self).__init__()
            self.weight = nn.Parameter(torch.ones(hidden_size))
            self.bias = nn.Parameter(torch.zeros(hidden_size))
            self.variance_epsilon = eps

        def forward(self, x):
            u = x.mean(-1, keepdim=True)
            s = (x - u).pow(2).mean(-1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.variance_epsilon)
            return self.weight * x + self.bias
        

class BertForSequenceClassification(nn.Module):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.
    Params:
        `config`: a BertConfig class instance with the configuration to build a new model.
        `num_labels`: the number of classes for the classifier. Default = 2.
    Inputs:
        `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length]
            with the word token indices in the vocabulary. Items in the batch should begin with the special "CLS" token. (see the tokens preprocessing logic in the scripts
            `extract_features.py`, `run_classifier.py` and `run_squad.py`)
        `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length] with the token
            types indices selected in [0, 1]. Type 0 corresponds to a `sentence A` and type 1 corresponds to
            a `sentence B` token (see BERT paper for more details).
        `attention_mask`: an optional torch.LongTensor of shape [batch_size, sequence_length] with indices
            selected in [0, 1]. It's a mask to be used if the input sequence length is smaller than the max
            input sequence length in the current batch. It's the mask that we typically use for attention when
            a batch has varying length sentences.
        `labels`: labels for the classification output: torch.LongTensor of shape [batch_size]
            with indices selected in [0, ..., num_labels].
    Outputs:
        if `labels` is not `None`:
            Outputs the CrossEntropy classification loss of the output with the labels.
        if `labels` is `None`:
            Outputs the classification logits of shape [batch_size, num_labels].
    Example usage:
    ```python
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
    num_labels = 2
    model = BertForSequenceClassification(config, num_labels)
    logits = model(input_ids, token_type_ids, input_mask)
    ```
    """
    def __init__(self, num_labels=2):
        super(BertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        nn.init.xavier_normal_(self.classifier.weight)
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        return logits
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [101]:
from pytorch_pretrained_bert import BertConfig

config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

num_labels = 2
model = BertForSequenceClassification(num_labels)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(zz)])

logits = model(tokens_tensor)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmppto2luuc
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



In [102]:
logits

tensor([[-1.3490,  1.2298]], grad_fn=<AddmmBackward>)

In [0]:
import pandas as pd

dat = pd.read_csv('bert_train_data.csv')
# dat = pd.read_csv('IMDB_Dataset_SCRATCH.csv')

# dat = pd.DataFrame()
# dat["review"] = raw_df["review"]
# dat["sentiment"] = raw_df["sentiment"]

In [104]:
dat.head()


,review,sentiment
0,deed reason may allah forgive u,positive
1,forest fire near la ronge sask canada,positive
2,resident asked shelter place notified officer ...,positive
3,people receive evacuation order california,positive
4,got sent photo ruby smoke pours school,positive


In [105]:
# dat.isnull().values.any()
# dat.isnull().sum().sum()
# dat.shape
dat.dropna(inplace=True)
dat.shape

(7610, 2)

In [106]:
dat.columns
dat.shape

(7610, 2)

In [107]:
zz = tokenizer.tokenize(dat.review[1])
z1z = tokenizer.tokenize(dat.review[3])

[zz,z1z ]

[['forest', 'fire', 'near', 'la', 'ron', '##ge', 'sas', '##k', 'canada'],
 ['people', 'receive', 'evacuation', 'order', 'california']]

In [0]:
zzz = tokenizer.convert_tokens_to_ids(zz)
zzzz = tokenizer.convert_tokens_to_ids(z1z)

In [109]:
zzz

[3224, 2543, 2379, 2474, 6902, 3351, 21871, 2243, 2710]

In [0]:

tokens_tensor = torch.tensor([zzz,zzz])


In [111]:
tokens_tensor

tensor([[ 3224,  2543,  2379,  2474,  6902,  3351, 21871,  2243,  2710],
        [ 3224,  2543,  2379,  2474,  6902,  3351, 21871,  2243,  2710]])

In [112]:
logits = model(tokens_tensor)
logits

tensor([[ 1.1570, -0.9083],
        [ 1.5735, -0.7240]], grad_fn=<AddmmBackward>)

In [113]:
import torch.nn.functional as F

F.softmax(logits,dim=1)

tensor([[0.8875, 0.1125],
        [0.9087, 0.0913]], grad_fn=<SoftmaxBackward>)

In [0]:
from sklearn.model_selection import train_test_split
X = dat['review']
y = dat['sentiment']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)


In [0]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()

y_train = pd.get_dummies(y_train).values.tolist()
y_test = pd.get_dummies(y_test).values.tolist()


In [116]:
print(y_train[:10])

[[1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1]]


In [0]:

max_seq_length = 256
class text_dataset(Dataset):
    def __init__(self,x_y_list, transform=None):
        
        self.x_y_list = x_y_list
        self.transform = transform
        
    def __getitem__(self,index):
        
        tokenized_review = tokenizer.tokenize(self.x_y_list[0][index])
        
        if len(tokenized_review) > max_seq_length:
            tokenized_review = tokenized_review[:max_seq_length]
            
        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_review)

        padding = [0] * (max_seq_length - len(ids_review))
        
        ids_review += padding
        
        assert len(ids_review) == max_seq_length
        
        #print(ids_review)
        ids_review = torch.tensor(ids_review)
        
        sentiment = self.x_y_list[1][index] # color        
        list_of_labels = [torch.from_numpy(np.array(sentiment))]
        
        
        return ids_review, list_of_labels[0]
    
    def __len__(self):
        return len(self.x_y_list[0])

In [118]:

batch_size = 16

train_lists = [X_train, y_train]
test_lists = [X_test, y_test]

training_dataset = text_dataset(x_y_list = train_lists )

test_dataset = text_dataset(x_y_list = test_lists )

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        sentiment_corrects = 0
        sentiment_wrongs = 0
        true_pos = 0
        true_neg = 0
        false_pos = 0
        false_neg = 0
        predicted_pos = 0

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            
            sentiment_corrects = 0
            sentiment_wrongs = 0
            true_pos = 0
            true_neg = 0
            false_pos = 0
            false_neg = 0
            predicted_pos = 0
            
            
            # Iterate over data.
            for inputs, sentiment in dataloaders_dict[phase]:
              # print("inputs: {}".format(print))
                #inputs = inputs
                #print(len(inputs),type(inputs),inputs)
                #inputs = torch.from_numpy(np.array(inputs)).to(device) 
                inputs = inputs.to(device) 

                sentiment = sentiment.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    #print(inputs)
                    outputs = model(inputs)

                    outputs = F.softmax(outputs,dim=1)
                    
                    loss = criterion(outputs, torch.max(sentiment.float(), 1)[1])
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)


                sentiment_corrects += torch.sum(torch.max(outputs, 1)[1] == torch.max(sentiment, 1)[1])
                sentiment_wrongs += torch.sum(torch.max(outputs, 1)[1] != torch.max(sentiment, 1)[1])
                predicted_pos += torch.sum(torch.max(outputs, 1)[1])

                tensor_product = torch.sum(torch.max(outputs, 1)[1] * torch.max(sentiment, 1)[1])
                true_pos += torch.sum(tensor_product)

                # print("outputs.shape: {}".format(outputs.shape))
                # print("sentiment.shape: {}".format(sentiment.shape))

                # print("       output: {}".format(torch.max(outputs, 1)[1]))
                # print("actual answer: {}".format(torch.max(sentiment, 1)[1]))
                # print("sentiment_corrects: {}".format(sentiment_corrects))
                # print("sentiment_wrongs: {}".format(sentiment_wrongs))
                # print("Total sum of example: {}".format(sentiment_wrongs + sentiment_corrects))

                # # print("\n\n\n")
                # # print("outputs: {}".format(outputs))
                # print("tensor_product: {}".format(tensor_product))
                # print("true_pos: {}".format(true_pos))

               

                
            epoch_loss = running_loss / dataset_sizes[phase]

            
            sentiment_acc = sentiment_corrects.double() / dataset_sizes[phase]

            print('{} total loss: {:.4f} '.format(phase,epoch_loss ))
            print('{} sentiment_acc: {:.4f}'.format(
                phase, sentiment_acc))

            if phase == 'val' and epoch_loss < best_loss:
                print('saving with loss of {}'.format(epoch_loss),
                      'improved over previous {}'.format(best_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'bert_model_test.pth')


        print()
        false_pos = predicted_pos - true_pos
        false_neg = sentiment_wrongs - false_pos
        true_neg = sentiment_corrects - true_pos

        precision = true_pos / (true_pos + false_pos)
        recall = true_pos / (true_pos + false_neg)
        f1_score = 2 * (precision * recall) / (precision + recall)

        print("true_pos: {}".format(true_pos))
        print("true_neg: {}".format(true_neg))
        print("false_pos: {}".format(false_pos))
        print("false_neg: {}".format(false_neg))

        print("\n\n")

        print("precision: {}".format(precision))
        print("recall: {}".format(recall))
        print("f1_score: {}".format(f1_score))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(float(best_loss)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [120]:
model.to(device)

#model.freeze_bert_encoder()
#model.classifier.weight.requires_grad = True
model

#    def freeze_bert_encoder(self):
#        for param in self.bert.parameters():
#            param.requires_grad = False
#    
#    def unfreeze_bert_encoder(self):
#        for param in self.bert.parameters():
#            param.requires_grad = True

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
lrlast = .001 # This is for the final FC layer
# lrlast = .005
lrmain = .00001 # This is for bert
optim1 = optim.Adam(
    [
        {"params":model.bert.parameters(),"lr": lrmain},
        {"params":model.classifier.parameters(), "lr": lrlast},
       
   ])

#optim1 = optim.Adam(model.parameters(), lr=0.001)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1
criterion = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)


In [122]:
model_ft1 = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)


starting
Epoch 0/9
----------


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train total loss: 0.6932 
train sentiment_acc: 0.5602
val total loss: 0.6996 
val sentiment_acc: 0.5506
saving with loss of 0.6995932856934456 improved over previous 100

true_pos: 0
true_neg: 419
false_pos: 0
false_neg: 342



precision: 4294967295
recall: 0
f1_score: 0
Epoch 1/9
----------
train total loss: 0.5786 
train sentiment_acc: 0.7248
val total loss: 0.5264 
val sentiment_acc: 0.7753
saving with loss of 0.52643028579153 improved over previous 0.6995932856934456

true_pos: 272
true_neg: 318
false_pos: 101
false_neg: 70



precision: 0
recall: 0
f1_score: 4294967295
Epoch 2/9
----------
train total loss: 0.5101 
train sentiment_acc: 0.7933
val total loss: 0.5083 
val sentiment_acc: 0.7924
saving with loss of 0.5082588557658152 improved over previous 0.52643028579153

true_pos: 262
true_neg: 341
false_pos: 78
false_neg: 80



precision: 0
recall: 0
f1_score: 4294967295
Epoch 3/9
----------
train total loss: 0.4990 
train sentiment_acc: 0.8058
val total loss: 0.5048 
val sentimen

In [123]:
x = [[1,0],[1,0],[0,1]]
a = torch.tensor(x)
b = [1,0]
b = torch.tensor(b)
b
# torch.max(a)
# torch.sum(a == b) 
torch.max(a, 1)[1]
# for i in range(a.shape[0]):
#   print(a[i])
c = torch.tensor([[0], [1], [1]])
d = torch.tensor([[1], [1], [0]])


torch.sum(c * d)
print(c)


tensor([[0],
        [1],
        [1]])


In [0]:
import numpy as np

def check_list_for_nan(input_list):
    '''
        Helper function, check if list contains any nan values
        Input: 
            - input_list (list)
        Return: 
            - array_has_nan (boolean)
    '''
    input_np_array = np.asarray(input_list)
    array_sum = np.sum(input_np_array)
    array_has_nan = np.isnan(array_sum)
    
    return array_has_nan

In [125]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   